In [22]:
from utils.Loader import import_pdf
from utils.Splitter import splitter
from openai import OpenAI
import yaml
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import os
from langchain.docstore.document import Document


In [32]:
N_RETRIEVED = 5
course_name = "python"
llm_engine = "gpt-3.5-turbo"

In [2]:
# importing the config file.
with open(r".\utils\Config.yml", "r", encoding="utf-8") as file:
    config = yaml.safe_load(file)

In [3]:
db_dir = os.path.join(os.getcwd(), "db")
if not os.path.exists(db_dir):
    os.mkdir(db_dir)

In [4]:
system_role_1_prompt = config["language_config"]["system_role_1"]
system_role_2_prompt = config["language_config"]["system_role_2"]
max_token_value = config["language_config"]["max_token"]

In [5]:
pdfs = import_pdf([r"python.pdf"])

loading the pdfs...


In [6]:
chunks = splitter(pdfs)

splitting document...


In [23]:
documents = [Document(page_content=chunk) for chunk in chunks]


In [ ]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="OPENAI_API_KEY",
)

In [ ]:
embedding = OpenAIEmbeddings(openai_api_key="OPENAI_API_KEY")

In [ ]:
#It should work once
vectordb = Chroma.from_documents(
            documents = documents,
            embedding = embedding,
            persist_directory=db_dir
        )

In [ ]:
vector_database = Chroma(persist_directory=db_dir, embedding_function = embedding)

In [12]:
query = "What is python?"

In [25]:
docs = vector_database.similarity_search(query, k=N_RETRIEVED)

In [28]:
retrieved_contents = [str(d.page_content)+"\n\n" for d in docs]

In [33]:
prompt = "# User's question: \n" + query + "# Retrieved content number:\n" + str(retrieved_contents) + "# Name of the course: \n" + str(course_name)

In [34]:
response = client.chat.completions.create(
        model= llm_engine,
        messages=[
            {"role": "system", "content": system_role_1_prompt},
            {"role": "user", "content": prompt}
            ]
    )

In [35]:
print(response.choices[0].message.content.strip())

**Python** is a programming language that is easy to master and allows the addition of modules to perform a wide variety of tasks. It is a true object-oriented language developed in the early 1990s by Guido van Rossum. Python's small, clean syntax reflects its focus on being easy for beginners to learn while also being powerful enough for advanced users. Python can be used for creating graphical user interfaces, communicating with databases, processing image files, and more.
